# Kingdom Hearts Save Editor

In [ ]:
! python kh1se.py

Dash is running on http://0.0.0.0:8080/

 * Serving Flask app 'KH1 Save Editor'
 * Debug mode: on


In [1]:
from ctypes import *
import ipywidgets as widgets
import kh1codec
import os
import struct
import remi
import remi.gui as gui
from remi import start, App

## KH1

In [2]:
class KH1Character:
    """
    Class for representing the character struct.
    So in C/C++ I'd use a struct instead.
    The structure is 0x74 bytes long.
    """
    def __init__(self, name, data):
        self.name = name
        self.level = c_ubyte(data[0])
        self.hp = c_ubyte(data[1])
        self.maxhp = c_ubyte(data[2])
        self.mp = c_ubyte(data[3])
        self.maxmp = c_ubyte(data[4])
        self.maxap = c_ubyte(data[5])
        self.strength = c_ubyte(data[6])
        self.defense = c_ubyte(data[7])
        # data[0x08:0x18] is unknown, full of 0x64
        self.accessorycount = c_ubyte(data[0x18])
        self.accessories = (c_ubyte*8)(*data[0x19:0x21])
        self.itemcount = c_ubyte(data[0x21])
        self.items = (c_ubyte*8)(*data[0x22:0x2A])
        # data[0x2A:0x32] is unknown
        self.weapon = c_ubyte(data[0x32])
        # data[0x33:0x38] is unknown
        self.submp = c_ushort(int.from_bytes(data[0x38:0x3A][::-1])) # 30 or 0x1E means 1 MP
        # data[0x3A:0x3C]is unknown
        self.exp = c_uint(int.from_bytes(data[0x3C:0x40][::-1]))
        self.abilities = (c_ubyte*48)(*data[0x40:0x70]) # Pooh has unknown data here with invalid ability IDs.
        self.magic = c_ubyte(data[0x70])
        # data[0x71:0x73] is unknown


class KH1GummiBlock:
    def __init__(self, data):
        pass


class KH1GummiShip:
    def __init__(self, data):
        self.blockcount = c_ushort(int.from_bytes(data[0x00:0x02][::-1]))
        self.x = c_ushort(int.from_bytes(data[0x02:0x04][::-1]))
        self.y = c_ushort(int.from_bytes(data[0x04:0x06][::-1]))
        self.z = c_ushort(int.from_bytes(data[0x06:0x08][::-1]))
        self.transformpair = c_ushort(int.from_bytes(data[0x08:0x0A][::-1]))
        self.name = bytearray(data[0x4C:0x56])


In [3]:
class KH1:
    def __init__(self, slot=0, fm=False):
        self.dicts()
        if slot != 0:
            self.fm = fm
            if self.fm:
                self.filename = "BISLPS-25198-" + f"{slot:02d}"
            else:
                self.filename = "BASLUS-20370-" + f"{slot:02d}"
            if os.path.exists(os.path.join(self.filename, self.filename)):
                with open(os.path.join(self.filename, self.filename), "rb") as file:
                    self.data = (c_ubyte*0x16C00)(*file.read())
            else:
                with open(self.filename, "rb") as file:
                    self.data = (c_ubyte*0x16C00)(*file.read())
            self.__parse_data(self.data)
            self.sysdata = None
            if os.path.exists(os.path.join(self.filename, "system.bin")):
                with open(os.path.join(self.filename, "system.bin"), "rb") as sysfile:
                    self.sysdata = (c_ubyte*0x400)(*sysfile.read())
                    # Playtime in seconds * 60 but possibly in seconds * 50 in PAL versions
                    self.playtime = c_uint(int.from_bytes(self.sysdata[0x10:0x14][::-1]))

    def __parse_data(self, data):
        # For FM the currently loaded save file starts at 0x3F8380 in the memory according to the RetroAchievements code notes.
        # For vanilla USA it starts at 0x3F1C90.
        # For vanilla JP it starts at 0x3F2080.
        self.header = c_uint(int.from_bytes(data[0x00:0x04:-1])) # an unnecessary header; 4 in vanilla, 5 in FM
        # self.characters = data[0x04:0x048C]
        self.sora = KH1Character("Sora", data[0x04:0x04+0x74])
        self.donald = KH1Character("Donald", data[0x04+0x74:0x04+2*0x74])
        self.goofy = KH1Character("Goofy", data[0x04+2*0x74:0x04+3*0x74])
        self.tarzan = KH1Character("Tarzan", data[0x04+3*0x74:0x04+4*0x74])
        self.pooh = KH1Character("Winnie the Pooh", data[0x04+4*0x74:0x04+5*0x74])
        self.aladdin = KH1Character("Aladdin", data[0x04+5*0x74:0x04+6*0x74])
        self.ariel = KH1Character("Ariel", data[0x04+6*0x74:0x04+7*0x74])
        self.jack = KH1Character("Jack Skellington", data[0x04+7*0x74:0x04+8*0x74])
        self.peterpan = KH1Character("Peter Pan", data[0x04+8*0x74:0x04+9*0x74])
        self.beast = KH1Character("Beast", data[0x04+9*0x74:0x04+10*0x74])
        self.characters = [self.sora, self.donald, self.goofy,
                           self.tarzan, self.pooh, self.aladdin,
                           self.ariel, self.jack, self.peterpan, self.beast]
        self.path = c_ubyte(data[0x048C])
        self.curve = c_ubyte(data[0x048D])
        self.party = (c_ubyte*4)(*data[0x048E:0x0492])
        self.magiclevels = (c_ubyte*7)(*data[0x0492:0x0499])
        self.inventory = (c_ubyte*256)(*data[0x0499:0x0599])
        self.shared_abilities = (c_ubyte*48)(*data[0x0599:0x05C9])
        # data[0x05C9:0x05CC] is unknown.
        self.di_chest_flag = c_ubyte(data[0x05CC])
        
        self.summons = (c_ubyte*6)(*data[0x07D0:0x07D6])
        # data[0x07D6:0x07D8] is unknown.
        self.heartless = (c_ushort*36)(*struct.unpack("<36H", bytearray(data[0x07D8:0x0820])))
        self.shortcuts = (c_ubyte*3)(*data[0x082C:0x082F])
        self.cure_on_friends = c_ushort(int.from_bytes(data[0x0836:0x0838][::-1]))
        self.heartless_killed = c_ushort(int.from_bytes(data[0x083E:0x0840][::-1]))
        self.deflected = c_ushort(int.from_bytes(data[0x0844:0x0846][::-1]))
        self.item_usage = c_ushort(int.from_bytes(data[0x0848:0x084A][::-1]))
        self.hits = c_ushort(int.from_bytes(data[0x084A:0x084C][::-1]))
        self.friend_ko = c_ushort(int.from_bytes(data[0x084C:0x084E][::-1]))
        self.deaths = c_ushort(int.from_bytes(data[0x084E:0x0850][::-1]))

        self.curcup = c_ubyte(data[0x0F26])
        self.philcup = c_ubyte(data[0x0F36])
        self.pegasuscup = c_ubyte(data[0x0F37])
        self.herculescup = c_ubyte(data[0x0F38])
        self.hadescup = c_ubyte(data[0x0F39])
        self.platinummatch = c_ubyte(data[0x0F6A])

        self.sorawins = c_ushort(int.from_bytes(data[0x1036:0x1038][::-1]))
        self.rikuwins = c_ushort(int.from_bytes(data[0x1038:0x103A][::-1]))

        self.slides = (c_ubyte*6)(*data[0x1207:0x120D])
        self.slides_watched = c_ubyte(data[0x1212])

        self.world_progresses = (c_ubyte*20)(*data[0x1500:0x1514])
        
        self.raft = bytearray(data[0x16D1:0x16DB])

        self.boss_journal = (c_ubyte*4)(*data[0x16F6:0x16FA])
        # data[0x16FA:0x1703] is unknown
        self.dalmatians = (c_ubyte*13)(*data[0x1703:0x1710])

        self.reports = (c_ubyte*2)(*data[0x19C0:0x19C2])
        self.journal_unlock = c_ubyte(data[0x19C4]) # bit index 3, 0x1F for completed game so needs further investigation

        self.trinity_unlock = c_ubyte(data[0x1C1B])
        self.trinity_count = (c_ubyte*6)(*data[0x1C66:0x1C6C]) # Jump, Unused, Charge, Ladder, Push, Detect
        
        self.world_statuses = (c_ubyte*15)(*data[0x1EF0:0x1EFF])
        self.landingpoints = (c_ubyte*15)(*data[0x1EFF:0x1F0E])
        
        self.world = c_uint(int.from_bytes(data[0x2040:0x2044][::-1]))
        self.room = c_uint(int.from_bytes(data[0x2044:0x2048][::-1]))
        self.flag = c_uint(int.from_bytes(data[0x2048:0x204C][::-1]))

        self.GUMI = bytearray(data[0x2400:0x2404]).decode() # ASCII string "GUMI"
        # data[0x2404] seems to be a version code, 0 for vanilla and 1 for FM, needs further investigation.
        self.gummitutorial = c_ubyte(data[0x2405]) # might be an int
        # data[0x2409:0x2410] is [1, 2, 3, 4, 5, 6, 7] for me
        self.selectedship = c_ubyte(data[0x2410])
        # self.gummiships = data[0x241C:0xBE7C], based on the start offsets of each ship
        # which I've confirmed but the last ship overlaps with the 1st 4 blocks which are also confirmed.
        self.gummiships = [KH1GummiShip(data[0x241C+i*0x0F70:0x241C+(i+1)*0x0F70]) for i in range(10)]
        self.gummiblocks = (c_ubyte*107)(*data[0xBE78:0xBEE3])

        self.gummi_decelerate = c_uint(int.from_bytes(data[0xBF01:0xBF05][::-1]))
        self.gummi_accelerate = c_uint(int.from_bytes(data[0xBF05:0xBF09][::-1]))
        self.gummi_transform = c_uint(int.from_bytes(data[0xBF09:0xBF0D][::-1]))
        self.gummi_scannon = c_uint(int.from_bytes(data[0xBF0D:0xBF11][::-1]))
        self.gummi_mcannon = c_uint(int.from_bytes(data[0xBF11:0xBF15][::-1]))
        self.gummi_lcannon = c_uint(int.from_bytes(data[0xBF15:0xBF19][::-1]))
        self.gummi_slaser = c_uint(int.from_bytes(data[0xBF19:0xBF1D][::-1]))
        self.gummi_mlaser = c_uint(int.from_bytes(data[0xBF1D:0xBF21][::-1]))
        self.gummi_llaser = c_uint(int.from_bytes(data[0xBF21:0xBF25][::-1]))
        
        self.autolock = c_uint(int.from_bytes(data[0x16400:0x16404][::-1]))
        self.targetlock = c_uint(int.from_bytes(data[0x16404:0x16408][::-1]))
        self.camera = c_uint(int.from_bytes(data[0x16408:0x1640C][::-1]))
        # data[0x1640C:0x16410] is unknown
        self.vibration = c_uint(int.from_bytes(data[0x16410:0x16414][::-1]))
        self.sound = c_uint(int.from_bytes(data[0x16414:0x16418][::-1]))
        self.datainstall = c_uint(int.from_bytes(data[0x16418:0x1641C][::-1])) # JP/FM
        self.difficulty = c_uint(int.from_bytes(data[0x16418:0x1641C][::-1])) # USA/EU
        self.munny = c_uint(int.from_bytes(data[0x1641C:0x16420][::-1]))

        # Final Mix stuff
        if self.fm:
            self.fmheartless = (c_ushort*15)(*struct.unpack("<15H", bytearray(data[0x0820:0x083E])))
            self.shortcuts = (c_ubyte*3)(*data[0x0844:0x0847])
            self.cure_on_friends = c_ushort(int.from_bytes(data[0x084E:0x0850][::-1]))
            self.heartless_killed = c_ushort(int.from_bytes(data[0x0856:0x0858][::-1]))
            self.deflected = c_ushort(int.from_bytes(data[0x085C:0x085E][::-1]))
            self.item_usage = c_ushort(int.from_bytes(data[0x0860:0x0862][::-1]))
            self.hits = c_ushort(int.from_bytes(data[0x0862:0x0864][::-1]))
            self.friend_ko = c_ushort(int.from_bytes(data[0x0864:0x0868][::-1]))
            self.deaths = c_ushort(int.from_bytes(data[0x0868:0x086A][::-1]))
            self.xemnas = c_ubyte(data[0x1118])
            self.gummi_decelerate = c_uint(int.from_bytes(data[0xBF41:0xBF45][::-1]))
            self.gummi_accelerate = c_uint(int.from_bytes(data[0xBF45:0xBF49][::-1]))
            self.gummi_transform = c_uint(int.from_bytes(data[0xBF49:0xBF4D][::-1]))
            self.gummi_scannon = c_uint(int.from_bytes(data[0xBF4D:0xBF51][::-1]))
            self.gummi_mcannon = c_uint(int.from_bytes(data[0xBF51:0xBF55][::-1]))
            self.gummi_lcannon = c_uint(int.from_bytes(data[0xBF55:0xBF59][::-1]))
            self.gummi_slaser = c_uint(int.from_bytes(data[0xBF59:0xBF5D][::-1]))
            self.gummi_mlaser = c_uint(int.from_bytes(data[0xBF5D:0xBF61][::-1]))
            self.gummi_llaser = c_uint(int.from_bytes(data[0xBF61:0xBF65][::-1]))
            self.difficulty = c_uint(int.from_bytes(data[0x1642C:0x16430][::-1]))

    def __save_characters(self):
        i = 0
        for c in self.characters:
            self.data[0x04+i*0x74+0x00] = c.level
            self.data[0x04+i*0x74+0x01] = c.hp
            self.data[0x04+i*0x74+0x02] = c.maxhp
            self.data[0x04+i*0x74+0x03] = c.mp
            self.data[0x04+i*0x74+0x04] = c.maxmp
            self.data[0x04+i*0x74+0x05] = c.maxap
            self.data[0x04+i*0x74+0x06] = c.strength
            self.data[0x04+i*0x74+0x07] = c.defense
            self.data[0x04+i*0x74+0x18] = c.accessorycount
            self.data[0x04+i*0x74+0x19:0x04+i*0x74+0x21] = bytearray(c.accessories)
            self.data[0x04+i*0x74+0x21] = c.itemcount
            self.data[0x04+i*0x74+0x22:0x04+i*0x74+0x2A] = bytearray(c.items)
            i += 1
        
    def save(self):
        if self.sysdata is not None:
            self.sysdata[0x10:0x14] = bytearray(self.playtime)
        self.__save_characters()
        self.data[0x048D] = self.curve
        self.data[0x048E:0x0492] = bytearray(self.party)
        self.data[0x049A:0x0599] = bytearray(self.inventory)
        self.data[0x0599:0x05C9] = bytearray(self.shared_abilities)
        
        self.data[0x1036:0x1038] = bytearray(self.sorawins)
        self.data[0x1038:0x103A] = bytearray(self.rikuwins)

        self.data[0x16D1:0x16DB] = self.raft
        self.data[0x2040:0x2044] = bytearray(self.world)
        self.data[0x2044:0x2048] = bytearray(self.room)
        self.data[0x2048:0x204C] = bytearray(self.flag)
        self.data[0x1641C:0x16420] = bytearray(self.munny)
        
        if self.fm:
            self.data[0x0844:0x0847] = bytearray(self.shortcuts)
            self.data[0x1642C] = self.difficulty
        else:
            self.data[0x082C:0x082F] = bytearray(self.shortcuts)
            self.data[0x16418] = self.difficulty

    def dicts(self):
        self.character_dict = {"Sora": 0,
                               "Donald": 1,
                               "Goofy": 2,
                               "Tarzan": 3,
                               "Winnie the Pooh": 4,
                               "Aladdin": 5,
                               "Ariel": 6,
                               "Jack Skellington": 7,
                               "Peter Pan": 8,
                               "Beast": 9,
                               "None": 255}
        self.ability_dict = {"Empty": 0x00,
                             "High Jump": 0x01,
                             "Mermaid Kick": 0x02,
                             "Glide": 0x03,
                             "Superglide": 0x04,
                             "Treasure Magnet": 0x05,
                             "Combo Plus": 0x06,
                             "Air Combo Plus": 0x07,
                             "Critical Plus": 0x08,
                             "Second Wind": 0x09,
                             "Scan": 0x0A,
                             "Sonic Blade": 0x0B,
                             "Ars Arcanum": 0x0C,
                             "Strike Raid": 0x0D,
                             "Ragnarok": 0x0E,
                             "Trinity Limit": 0x0F,
                             "Cheer": 0x10,
                             "Vortex": 0x11,
                             "Aerial Sweep": 0x12,
                             "Counterattack": 0x13,
                             "Blitz": 0x14,
                             "Guard": 0x15,
                             "Dodge Roll": 0x16,
                             "MP Haste": 0x17,
                             "MP Rage": 0x18,
                             "Second Chance": 0x19,
                             "Berserk": 0x1A,
                             "Jackpot": 0x1B,
                             "Lucky Strike": 0x1C,
                             "Charge": 0x1D,
                             "Rocket": 0x1E,
                             "Tornado": 0x1F,
                             "MP Gift": 0x20,
                             "Raging Boar": 0x21,
                             "Asp’s Bite": 0x22,
                             "Healing Herb": 0x23,
                             "Wind Armor": 0x24,
                             "Crescent": 0x25,
                             "Sandstorm": 0x26,
                             "Applause!": 0x27,
                             "Blazing Fury": 0x28,
                             "Icy Terror": 0x29,
                             "Bolts of Sorrow": 0x2A,
                             "Ghostly Scream": 0x2B,
                             "Hummingbird": 0x2C,
                             "Time-Out": 0x2D,
                             "Storm’s Eye": 0x2E,
                             "Ferocious Lunge": 0x2F,
                             "Furious Bellow": 0x30,
                             "Spiral Wave": 0x31,
                             "Thunder Potion": 0x32,
                             "Cure Potion": 0x33,
                             "Aero Potion": 0x34,
                             # FM exclusives
                             "Slapshot": 0x35,
                             "Sliding Dash": 0x36,
                             "Hurricane Blast": 0x37,
                             "Ripple Drive": 0x38,
                             "Stun Impact": 0x39,
                             "Gravity Break": 0x3A,
                             "Zantetsuken": 0x3B,
                             "Tech Boost": 0x3C,
                             "Encounter Plus": 0x3D,
                             "Leaf Bracer": 0x3E,
                             "Evolution": 0x3F,
                             # Remix exclusives
                             "EXP Zero": 0x40,
                             "Combo Master": 0x41}
        self.item1_dict = {"Empty": 0x00,
                           "Potion": 0x01,
                           "Hi-Potion": 0x02,
                           "Ether": 0x03,
                           "Elixir": 0x04,
                           "Unused 0x05": 0x05,
                           "Mega-Potion": 0x06,
                           "Mega-Ether": 0x07,
                           "Megalixir": 0x08,
                           "Fury Stone": 0x09,
                           "Power Stone": 0x0A,
                           "Energy Stone": 0x0B,
                           "Blazing Stone": 0x0C,
                           "Frost Stone": 0x0D,
                           "Lightning Stone": 0x0E,
                           "Dazzling Stone": 0x0F,
                           "Stormy Stone": 0x10,
                           "Protect Chain": 0x11,
                           "Protera Chain": 0x12,
                           "Protega Chain": 0x13,
                           "Fire Ring": 0x14,
                           "Fira Ring": 0x15,
                           "Firaga Ring": 0x16,
                           "Blizzard Ring": 0x17,
                           "Blizzara Ring": 0x18,
                           "Blizzaga Ring": 0x19,
                           "Thunder Ring": 0x1A,
                           "Thundara Ring": 0x1B,
                           "Thundaga Ring": 0x1C,
                           "Ability Stud": 0x1D,
                           "Guard Earring": 0x1E,
                           "Master Earring": 0x1F,
                           "Chaos Ring": 0x20,
                           "Dark Ring": 0x21,
                           "Element Ring": 0x22,
                           "Three Stars": 0x23,
                           "Power Chain": 0x24,
                           "Golem Chain": 0x25,
                           "Titan Chain": 0x26,
                           "Energy Bangle": 0x27,
                           "Angel Bangle": 0x28,
                           "Gaia Bangle": 0x29,
                           "Magic Armlet": 0x2A,
                           "Rune Armlet": 0x2B,
                           "Atlas Armlet": 0x2C,
                           "Heartguard": 0x2D,
                           "Ribbon": 0x2E,
                           "Crystal Crown": 0x2F,
                           "Brave Warrior": 0x30,
                           "Ifrit's Horn": 0x31,
                           "Inferno Band": 0x32,
                           "White Fang": 0x33,
                           "Ray of Light": 0x34,
                           "Holy Circlet": 0x35,
                           "Raven's Claw": 0x36,
                           "Omega Arts": 0x37,
                           "EXP Earring": 0x38,
                           "Unused 0x39": 0x39,
                           "EXP Ring": 0x3A,
                           "EXP Bracelet": 0x3B,
                           "EXP Necklace": 0x3C,
                           "Firagun Band": 0x3D,
                           "Blizzagun Band": 0x3E,
                           "Thundagun Band": 0x3F,
                           "Ifrit Belt": 0x40,
                           "Shiva Belt": 0x41,
                           "Ramuh Belt": 0x42,
                           "Moogle Badge": 0x43,
                           "Cosmic Arts": 0x44,
                           "Royal Crown": 0x45,
                           "Prime Cap": 0x46,
                           "Obsidian Ring": 0x47,
                           "Unused 0x48": 0x48,
                           "Unused 0x49": 0x49,
                           "Unused 0x4A": 0x4A,
                           "Unused 0x4B": 0x4B,
                           "Unused 0x4C": 0x4C,
                           "Unused 0x4D": 0x4D,
                           "Unused 0x4E": 0x4E,
                           "Unused 0x4F": 0x4F,
                           "Unused 0x50": 0x50}
        self.weapon_dict = {"Empty": 0x00,
                            "Kingdom Key": 0x51,
                            "Dream Sword": 0x52,
                            "Dream Shield": 0x53,
                            "Dream Rod": 0x54,
                            "Wooden Sword": 0x55,
                            "Jungle King": 0x56,
                            "Three Wishes": 0x57,
                            "Fairy Harp": 0x58,
                            "Pumpkinhead": 0x59,
                            "Crabclaw": 0x5A,
                            "Divine Rose": 0x5B,
                            "Spellbinder": 0x5C,
                            "Olympia": 0x5D,
                            "Lionheart": 0x5E,
                            "Metal Chocobo": 0x5F,
                            "Oathkeeper": 0x60,
                            "Oblivion": 0x61,
                            "Lady Luck": 0x62,
                            "Wishing Star": 0x63,
                            "Ultima Weapon": 0x64,
                            "Diamond Dust": 0x65,
                            "One-Winged Angel": 0x66,
                            "Mage's Staff": 0x67,
                            "Morning Star": 0x68,
                            "Shooting Star": 0x69,
                            "Magus Staff": 0x6A,
                            "Wisdom Staff": 0x6B,
                            "Warhammer": 0x6C,
                            "Silver Mallet": 0x6D,
                            "Grand Mallet": 0x6E,
                            "Lord Fortune": 0x6F,
                            "Violetta": 0x70,
                            "Dream Rod (Donald)": 0x71,
                            "Save the Queen": 0x72,
                            "Wizard’s Relic": 0x73,
                            "Meteor Strike": 0x74,
                            "Fantasista": 0x75,
                            "Unused (Donald)": 0x76,
                            "Knight’s Shield": 0x77,
                            "Mythril Shield": 0x78,
                            "Onyx Shield": 0x79,
                            "Stout Shield": 0x7A,
                            "Golem Shield": 0x7B,
                            "Adamant Shield": 0x7C,
                            "Smasher": 0x7D,
                            "Gigas Fist": 0x7E,
                            "Genji Shield": 0x7F,
                            "Herc’s Shield": 0x80,
                            "Dream Shield (Goofy)": 0x81,
                            "Save the King": 0x82,
                            "Defender": 0x83,
                            "Mighty Shield": 0x84,
                            "Seven Elements": 0x85,
                            "Unused (Goofy)": 0x86,
                            "Spear (Tarzan)": 0x87,
                            "No Weapon (Pooh)": 0x88,
                            "Sword (Aladdin)": 0x89,
                            "No Weapon (Ariel)": 0x8A,
                            "No Weapon (Jack)": 0x8B,
                            "Dagger (Peter Pan)": 0x8C,
                            "Claws (Beast)": 0x8D}
        self.item2_dict = {"Tent": 0x8E,
                           "Camping Set": 0x8F,
                           "Cottage": 0x90,
                           "Unused 0x91": 0x91,
                           "Unused 0x92": 0x92,
                           "Unused 0x93": 0x93,
                           "Unused 0x94": 0x94,
                           "Ansem's Report 11": 0x95,
                           "Ansem's Report 12": 0x96,
                           "Ansem's Report 13": 0x97,
                           "Power Up": 0x98,
                           "Defense Up": 0x99,
                           "AP Up": 0x9A,
                           "Serenity Power": 0x9B,
                           "Dark Matter": 0x9C,
                           "Mythril Stone": 0x9D,
                           "Fire Arts": 0x9E,
                           "Blizzard Arts": 0x9F,
                           "Thunder Arts": 0xA0,
                           "Cure Arts": 0xA1,
                           "Gravity Arts": 0xA2,
                           "Stop Arts": 0xA3,
                           "Aero Arts": 0xA4,
                           "Shiitank Rank": 0xA5,
                           "Matsutake Rank": 0xA6,
                           "Mystery Mold": 0xA7,
                           "Ansem's Report 1": 0xA8,
                           "Ansem's Report 2": 0xA9,
                           "Ansem's Report 3": 0xAA,
                           "Ansem's Report 4": 0xAB,
                           "Ansem's Report 5": 0xAC,
                           "Ansem's Report 6": 0xAD,
                           "Ansem's Report 7": 0xAE,
                           "Ansem's Report 8": 0xAF,
                           "Ansem's Report 9": 0xB0,
                           "Ansem's Report 10": 0xB1,
                           "Khama Vol. 8": 0xB2,
                           "Salegg Vol. 6": 0xB3,
                           "Azal Vol. 3": 0xB4,
                           "Mava Vol. 3": 0xB5,
                           "Mava Vol. 6": 0xB6,
                           "Theon Vol. 6": 0xB7,
                           "Nahara Vol. 5": 0xB8,
                           "Hafet Vol. 4": 0xB9,
                           "Empty Bottle": 0xBA,
                           "Old Book": 0xBB,
                           "Emblem Piece": 0xBC,
                           "Emblem Piece 2": 0xBD,
                           "Emblem Piece 3": 0xBE,
                           "Emblem Piece 4": 0xBF,
                           "Log": 0xC0,
                           "Cloth": 0xC1,
                           "Rope": 0xC2,
                           "Seagull Egg": 0xC3,
                           "Fish": 0xC4,
                           "Mushroom": 0xC5,
                           "Coconut": 0xC6,
                           "Drinking Water": 0xC7,
                           "Navi-G Piece 1": 0xC8,
                           "Navi-G Piece 2": 0xC9,
                           "Navi-Gummi 1": 0xCA,
                           "Navi-G Piece 3": 0xCB,
                           "Navi-G Piece 4": 0xCC,
                           "Navi-Gummi 2": 0xCD,
                           "Watergleam": 0xCE,
                           "Naturespark": 0xCF,
                           "Fireglow": 0xD0,
                           "Earthshine": 0xD1,
                           "Crystal Trident": 0xD2,
                           "Postcard": 0xD3,
                           "Torn Page 1": 0xD4,
                           "Torn Page 2": 0xD5,
                           "Torn Page 3": 0xD6,
                           "Torn Page 4": 0xD7,
                           "Torn Page 5": 0xD8,
                           "Slide 1": 0xD9,
                           "Slide 2": 0xDA,
                           "Slide 3": 0xDB,
                           "Slide 4": 0xDC,
                           "Slide 5": 0xDD,
                           "Slide 6": 0xDE,
                           "Footprints": 0xDF,
                           "Claw Marks": 0xE0,
                           "Stench": 0xE1,
                           "Antenna": 0xE2,
                           "Forget-Me-Not": 0xE3,
                           "Jack-In-The-Box": 0xE4,
                           "Entry Pass": 0xE5,
                           "Hero License": 0xE6,
                           "Pretty Stone": 0xE7,
                           "Unused 0xE8": 0xE8,
                           "Lucid Shard": 0xE9,
                           "Lucid Gem": 0xEA,
                           "Lucid Crystal": 0xEB,
                           "Spirit Shard": 0xEC,
                           "Spirit Gem": 0xED,
                           "Power Shard": 0xEE,
                           "Power Gem": 0xEF,
                           "Power Crystal": 0xF0,
                           "Blaze Shard": 0xF1,
                           "Blaze Gem": 0xF2,
                           "Frost Shard": 0xF3,
                           "Frost Gem": 0xF4,
                           "Thunder Shard": 0xF5,
                           "Thunder Gem": 0xF6,
                           "Shiny Crystal": 0xF7,
                           "Bright Shard": 0xF8,
                           "Bright Gem": 0xF9,
                           "Bright Crystal": 0xFA,
                           "Mystery Goo": 0xFB,
                           "Gale": 0xFC,
                           "Mythril Shard": 0xFD,
                           "Mythril": 0xFE,
                           "Orichalcum": 0xFF}
        self.item_dict = self.item1_dict | self.weapon_dict | self.item2_dict
        self.world_dict = {0x00: "Dive to the Heart",
                           0x01: "Destiny Islands",
                           0x02: "Disney Castle",
                           0x03: "Traverse Town",
                           0x04: "Wonderland",
                           0x05: "Deep Jungle",
                           0x06: "Hundred Acre Woods",
                           #0x07: "Unused 0x07",
                           0x08: "Agrabah",
                           0x09: "Atlantica",
                           0x0A: "Halloween Town",
                           0x0B: "Olympus Coliseum",
                           0x0C: "Monstro",
                           0x0D: "Neverland",
                           #0x0E: "Unused 0x0E",
                           0x0F: "Hollow Bastion",
                           0x10: "End of the World"}
        self.magicnames = ["Fire", "Blizzard", "Thunder", "Cure", "Gravity", "Stop", "Aero"]
        self.magicnames2 = ["Fira", "Blizzara", "Thundara", "Cura", "Gravira", "Stopra", "Aerora"]
        self.magicnames3 = ["Firaga", "Blizzaga", "Thundaga", "Curaga", "Graviga", "Stopga", "Aeroga"]
        self.heartlessnames = ["Soldier", "Shadow",
                               "Powerwild", "Bouncywild",
                               "Large Body", "Fat Bandit",
                               "Sea Neon", "Sheltering Zone",
                               "Bandit", "Pirate",
                               "Red Nocturne", "Blue Rhapsody", "Yellow Opera", "Green Requiem",
                               "Wizard", "Air Soldier",
                               "Pot Spider", "Barrel Spider", "Unused 19",
                               "Wight Knight", "Air Pirate", "Gargoyle", "Search Ghost",
                               "Aquatank", "Screwdriver",
                               "Unused 26", "Battleship",
                               "Darkball", "Invisible", "Behemoth",
                               "Wyvern", "Angel Star", "Defender",
                               "White Mushroom", "Black Fungus", "Rare Truffle"]
        self.fmheartlessnames = ["Soldier", "Shadow",
                                 "Powerwild", "Bouncywild",
                                 "Large Body", "Fat Bandit",
                                 "Sea Neon", "Sheltering Zone",
                                 "Bandit", "Pirate",
                                 "Red Nocturne", "Blue Rhapsody", "Yellow Opera", "Green Requiem",
                                 "Wizard", "Air Soldier",
                                 "Pot Spider", "Barrel Spider", "Pot Scorpion",
                                 "Wight Knight", "Air Pirate", "Gargoyle", "Search Ghost",
                                 "Aquatank", "Screwdriver",
                                 "Chimera", "Battleship",
                                 "Darkball", "Invisible", "Behemoth",
                                 "Wyvern", "Angel Star", "Defender",
                                 "White Mushroom", "Black Fungus", "Rare Truffle",
                                 "Unused 37", "Unused 38", "Unused 39",
                                 "Pink Agaricus", "Neoshadow", "Stealth Soldier", "Gigas Shadow",
                                 "Sniperwild", "Black Ballade",
                                 "Jet Balloon", "Unused 48", "Grand Ghost",
                                 "Destroyed Behemoth", "Arch Behemoth", "Sneak Army"]


### GUI

In [4]:
class KH1SaveEditor(App):
    def __init__(self, *args):
        super(KH1SaveEditor, self).__init__(*args)

    def open(self, b=None):
        self.kh1 = KH1(int(self.slot.get_value()), bool(self.fm.get_value()))
        global kh1
        kh1 = self.kh1
        self.__parse()

    def save(self, b=None):
        if hasattr(self, "kh1"):
            self.kh1.save()
        else:
            print("There's no loaded file!")

    def __encoding_callback(self, emitter, value):
        if hasattr(self, "kh1"):
            if not self.kh1.fm:
                self.datainstall.set_enabled(not bool(int(emitter.get_key())))
                self.difficulty.set_enabled(bool(int(emitter.get_key())))
            if hasattr(self, "raft"):
                self.raft.set_value(self.kh1.raft.decode(["kh1jp", "kh1us"][int(emitter.get_key())]))

    def __parse_playtime(self):
        if self.kh1.sysdata is not None:
            playtime = self.kh1.playtime.value // 60
            self.hours = gui.SpinBox(playtime // 3600, 0, 400 - 1, 1, width="60px")
            self.minutes = gui.SpinBox((playtime % 3600) // 60, 0, 59, 1, width="30px")
            self.seconds = gui.SpinBox((playtime % 3600) % 60, 0, 59, 1, width="30px")
            self.fraction = gui.SpinBox(self.kh1.playtime.value % 60, 0, 59, 1, width="30px")
            self.hours.onchange.do(lambda emitter, value: setattr(self.kh1.playtime, "value",
                                                         ((int(value) * 3600 +
                                                           int(self.minutes.get_value()) * 60 +
                                                           int(self.seconds.get_value())) * 60 +
                                                          int(self.fraction.get_value()))))
            self.minutes.onchange.do(lambda emitter, value: setattr(self.kh1.playtime, "value",
                                                           ((int(self.hours.get_value()) * 3600 +
                                                             int(value) * 60 +
                                                             int(self.seconds.get_value())) * 60 +
                                                            int(self.fraction.get_value()))))
            self.seconds.onchange.do(lambda emitter, value: setattr(self.kh1.playtime, "value",
                                                           ((int(self.hours.get_value()) * 3600 +
                                                             int(self.minutes.get_value()) * 60 +
                                                             int(value)) * 60 +
                                                            int(self.fraction.get_value()))))
            self.fraction.onchange.do(lambda emitter, value: setattr(self.kh1.playtime, "value",
                                                            ((int(self.hours.get_value()) * 3600 +
                                                              int(self.minutes.get_value()) * 60 +
                                                              int(self.seconds.get_value())) * 60 +
                                                             int(value))))
            self.playtime = gui.HBox([gui.Label("Playtime:", style={"width":"80px", "text-align":"left"}),
                                      self.hours, gui.Label(":"),
                                      self.minutes, gui.Label(":"),
                                      self.seconds, gui.Label(":"),
                                      self.fraction], width=f"{80+60+3*30+40}px", margin="5px")
        else:
            self.playtime = gui.Container()

    def __parse_general(self):
        self.__parse_playtime()
        self.curve = gui.DropDown(width="100px")
        self.curve.append(gui.DropDownItem("Dawn"), 0)
        self.curve.append(gui.DropDownItem("Midday"), 1)
        self.curve.append(gui.DropDownItem("Dusk"), 2)
        self.curve.set_key(self.kh1.curve.value)
        self.curve.onchange.do(lambda emitter, value: setattr(self.kh1.curve, "value", int(emitter.get_key())))
        curve = gui.HBox([gui.Label("Leveling Curve:", style={"width":"100px", "text-align":"left"}), self.curve], width="200px", margin="5px")
        self.path = gui.DropDown(width="100px")
        self.path.append(gui.DropDownItem("Warrior"), 0)
        self.path.append(gui.DropDownItem("Guardian"), 1)
        self.path.append(gui.DropDownItem("Mystic"), 2)
        self.path.set_key(self.kh1.path.value)
        self.path.onchange.do(lambda emitter, value: setattr(self.kh1.path, "value", int(emitter.get_key())))
        path = gui.HBox([gui.Label("Path:", style={"width":"100px", "text-align":"left"}), self.path], width="200px", margin="5px")
        self.general = gui.Container([self.playtime, curve, path])
        self.tabs.add_tab(self.general, "General")

    def __parse_char(self, c: KH1Character):
        weapon = gui.DropDown(width="160px")
        [weapon.append(gui.DropDownItem(k), v) for k, v in kh1.weapon_dict.items()]
        weapon.set_key(c.weapon.value)
        weapon.onchange.do(lambda emitter, value: setattr(c.weapon, "value", int(emitter.get_key())))
        weapon = gui.HBox([gui.Label("Weapon:", style={"width":"80px", "text-align":"left"}), weapon], width="240px", margin="5px")
        exp = gui.SpinBox(c.exp.value, 0, 999999, 1, width="160px")
        exp.onchange.do(lambda emitter, value: setattr(c.exp, "value", int(value)))
        exp = gui.HBox([gui.Label("EXP:", style={"width":"80px", "text-align":"left"}), exp], width="240px", margin="5px")
        level = gui.SpinBox(c.level.value, 1, 100, 1, width="160px")
        level.onchange.do(lambda emitter, value: setattr(c.level, "value", int(value)))
        level = gui.HBox([gui.Label("Level:", style={"width":"80px", "text-align":"left"}), level], width="240px", margin="5px")
        hp = gui.SpinBox(c.hp.value, 0, 255, 1, width="160px")
        hp.onchange.do(lambda emitter, value: setattr(c.hp, "value", int(value)))
        hp = gui.HBox([gui.Label("HP:", style={"width":"80px", "text-align":"left"}), hp], width="240px", margin="5px")
        maxhp = gui.SpinBox(c.maxhp.value, 0, 255, 1, width="160px")
        maxhp.onchange.do(lambda emitter, value: setattr(c.maxhp, "value", int(value)))
        maxhp = gui.HBox([gui.Label("Max HP:", style={"width":"80px", "text-align":"left"}), maxhp], width="240px", margin="5px")
        mp = gui.SpinBox(c.mp.value, 0, 255, 1, width="160px")
        mp.onchange.do(lambda emitter, value: (setattr(c.mp, "value", int(value)),
                                               setattr(submp1, "attr_max", value),
                                               setattr(submp1, "attr_value", value if int(submp1.get_value()) > int(value) else submp1.get_value())))
        mp = gui.HBox([gui.Label("MP:", style={"width":"80px", "text-align":"left"}), mp], width="240px", margin="5px")
        maxmp = gui.SpinBox(c.maxmp.value, 0, 255, 1, width="160px")
        maxmp.onchange.do(lambda emitter, value: setattr(c.maxmp, "value", int(value)))
        maxmp = gui.HBox([gui.Label("Max MP:", style={"width":"80px", "text-align":"left"}), maxmp], width="240px", margin="5px")
        submp1 = gui.SpinBox(c.submp.value // 30, 0, c.mp.value, 1, width="75px")
        submp2 = gui.SpinBox(c.submp.value % 30, 0, 29 if int(submp1.get_value()) < int(submp1.attr_max) else 0, 1, width="75px")
        submp1.onchange.do(lambda emitter, value: (setattr(c.submp, "value", int(value) * 30 + int(submp2.get_value())),
                                                   setattr(submp2, "attr_max", 29 if int(value) < int(emitter.attr_max) else 0),
                                                   setattr(submp2, "attr_value", 0 if int(value) == int(emitter.attr_max) else submp2.get_value())))
        submp2.onchange.do(lambda emitter, value: setattr(c.submp, "value", int(submp1.get_value()) * 30 + int(value)))
        submp = gui.HBox([gui.Label("Sub MP:", style={"width":"80px", "text-align":"left"}), submp1, gui.Label(":"), submp2], width="240px", margin="5px")
        maxap = gui.SpinBox(c.maxap.value, 0, 255, 1, width="160px")
        maxap.onchange.do(lambda emitter, value: setattr(c.maxap, "value", int(value)))
        maxap = gui.HBox([gui.Label("Max AP:", style={"width":"80px", "text-align":"left"}), maxap], width="240px", margin="5px")
        strength = gui.SpinBox(c.strength.value, 0, 255, 1, width="160px")
        strength.onchange.do(lambda emitter, value: setattr(c.strength, "value", int(value)))
        strength = gui.HBox([gui.Label("Strength:", style={"width":"80px", "text-align":"left"}), strength], width="240px", margin="5px")
        defense = gui.SpinBox(c.defense.value, 0, 255, 1, width="160px")
        defense.onchange.do(lambda emitter, value: setattr(c.defense, "value", int(value)))
        defense = gui.HBox([gui.Label("Defense:", style={"width":"80px", "text-align":"left"}), defense], width="240px", margin="5px")
        magic = gui.Container([gui.HBox([gui.Label(self.kh1.magicnames[i], style={"width":"80px", "text-align":"left"}),
                                         gui.CheckBox(checked=bool(c.magic.value & (1 << i)))]) for i in range(7)], width="100px", margin="5px")
        [list(list(magic.children.values())[i].children.values())[1].onchange.do(lambda emitter, value, idx=i: setattr(c.magic, "value", (c.magic.value | (1 << idx)) if value else (c.magic.value & ~(1 << idx)))) for i in range(7)]
        abilities1 = gui.Container([])
        abilities2 = gui.Container([])
        abilities = gui.HBox([abilities1, abilities2], style={"position":"relative", "top":"-430px", "left":"300px", "width":"360px", "margin":"5px"})
        for i in range(48):
            cb = gui.CheckBox(checked=c.abilities[i] & (1 << 7) == 0)
            cb.onchange.do(lambda emitter, value, idx=i: c.abilities.__setitem__(idx, c.abilities[idx] & ~(1 << 7) if value else c.abilities[idx] | (1 << 7)))
            dd = gui.DropDown(width="140px")
            [dd.append(gui.DropDownItem(k), v) for k, v in kh1.ability_dict.items()]
            dd.set_key(c.abilities[i] & ~(1 << 7))
            dd.onchange.do(lambda emitter, value, idx=i, cbi=cb: c.abilities.__setitem__(idx, int(emitter.get_key()) | (1 << 7) if not bool(cbi.get_value()) else int(emitter.get_key())))
            if i < 24:
                abilities1.append(gui.HBox([cb, dd], width="160px", margin="5px"))
            else:
                abilities2.append(gui.HBox([cb, dd], width="160px", margin="5px"))
        return gui.Container([weapon, exp, level, hp, maxhp, mp, maxmp, submp, maxap, strength, defense, magic, abilities], left="20px", margin="5px")

    def __parse_characters(self):
        self.characters = gui.TabBox()
        # I leave out Pooh because of his invalid abilities for now.
        [self.characters.add_tab(self.__parse_char(c), c.name) for c in self.kh1.characters if c.name != "Winnie the Pooh"]
        self.tabs.add_tab(self.characters, "Characters")

    def __parse_inventory(self):
        pass

    def __parse_journal(self):
        pass

    def __parse_config(self):
        self.autolock = gui.DropDown(width="100px")
        self.autolock.append(gui.DropDownItem("On"), 0)
        self.autolock.append(gui.DropDownItem("Off"), 1)
        self.autolock.set_key(self.kh1.autolock.value)
        self.autolock.onchange.do(lambda emitter, value: setattr(self.kh1.autolock, "value", int(emitter.get_key())))
        autolock = gui.HBox([gui.Label("Auto Lock:", style={"width":"100px", "text-align":"left"}), self.autolock], width="200px", margin="5px")
        
        self.targetlock = gui.DropDown(width="100px")
        self.targetlock.append(gui.DropDownItem("Auto"), 0)
        self.targetlock.append(gui.DropDownItem("Manual"), 1)
        self.targetlock.set_key(self.kh1.targetlock.value)
        self.targetlock.onchange.do(lambda emitter, value: setattr(self.kh1.targetlock, "value", int(emitter.get_key())))
        targetlock = gui.HBox([gui.Label("Target Lock:", style={"width":"100px", "text-align":"left"}), self.targetlock], width="200px", margin="5px")
        
        self.camera = gui.DropDown(width="100px")
        self.camera.append(gui.DropDownItem("Auto"), 0)
        self.camera.append(gui.DropDownItem("Manual"), 1)
        self.camera.set_key(self.kh1.camera.value)
        self.camera.onchange.do(lambda emitter, value: setattr(self.kh1.camera, "value", int(emitter.get_key())))
        camera = gui.HBox([gui.Label("Camera:", style={"width":"100px", "text-align":"left"}), self.camera], width="200px", margin="5px")
        
        self.vibration = gui.DropDown(width="100px")
        self.vibration.append(gui.DropDownItem("On"), 0)
        self.vibration.append(gui.DropDownItem("Off"), 1)
        self.vibration.set_key(self.kh1.vibration.value)
        self.vibration.onchange.do(lambda emitter, value: setattr(self.kh1.vibration, "value", int(emitter.get_key())))
        vibration = gui.HBox([gui.Label("Vibration:", style={"width":"100px", "text-align":"left"}), self.vibration], width="200px", margin="5px")
        
        self.sound = gui.DropDown(width="100px")
        self.sound.append(gui.DropDownItem("Stereo"), 0)
        self.sound.append(gui.DropDownItem("Mono"), 1)
        self.sound.set_key(self.kh1.sound.value)
        self.sound.onchange.do(lambda emitter, value: setattr(self.kh1.sound, "value", int(emitter.get_key())))
        sound = gui.HBox([gui.Label("Sound:", style={"width":"100px", "text-align":"left"}), self.sound], width="200px", margin="5px")
        
        self.datainstall = gui.DropDown(width="100px")
        self.datainstall.append(gui.DropDownItem("DVD-ROM"), 0)
        self.datainstall.append(gui.DropDownItem("Hard Drive"), 1)
        self.datainstall.set_key(self.kh1.datainstall.value)
        self.datainstall.onchange.do(lambda emitter, value: setattr(self.kh1.datainstall, "value", int(emitter.get_key())))
        datainstall = gui.HBox([gui.Label("Data Install:", style={"width":"100px", "text-align":"left"}), self.datainstall], width="200px", margin="5px")
        
        self.difficulty = gui.DropDown(width="100px")
        if self.kh1.fm:
            self.difficulty.append(gui.DropDownItem("Beginner"), 0)
            self.difficulty.append(gui.DropDownItem("Standard"), 1)
            self.difficulty.append(gui.DropDownItem("Proud"), 2)
        else:
            self.difficulty.append(gui.DropDownItem("Normal"), 0)
            self.difficulty.append(gui.DropDownItem("Expert"), 1)
        self.difficulty.set_key(self.kh1.difficulty.value)
        self.difficulty.onchange.do(lambda emitter, value: setattr(self.kh1.difficulty, "value", int(emitter.get_key())))
        self.encoding.onchange.do(self.__encoding_callback)
        # Initial disabling
        self.__encoding_callback(self.encoding, self.encoding.get_value())
        difficulty = gui.HBox([gui.Label("Difficulty:", style={"width":"100px", "text-align":"left"}), self.difficulty], width="200px", margin="5px")
        
        self.config = gui.Container([autolock, targetlock, camera, vibration, sound, datainstall, difficulty], left="20px", margin="5px")
        self.tabs.add_tab(self.config, "Config")

    def __parse_misc(self):
        self.misc = gui.Container()
        self.raft = gui.Input("", self.kh1.raft.decode(["kh1jp", "kh1us"][int(self.encoding.get_key())]), margin="5px")
        self.misc.append(self.raft)
        self.tabs.add_tab(self.misc, "Misc")        

    def __parse(self):
        self.tabroot.empty()
        self.tabs = gui.TabBox()
        self.__parse_general()
        self.__parse_characters()
        self.__parse_inventory()
        self.__parse_journal()
        self.__parse_config()
        self.__parse_misc()
        self.tabroot.append(self.tabs)

    def menu_setup(self):
        self.slot = gui.SpinBox(1, 1, 99, 1, width="50px", heigth="auto", margin="5px")
        slot = gui.HBox([gui.Label("Slot:"), self.slot], width="70px")
        slot.style["position"] = "absolute"
        slot.style["top"] = "20px"
        slot.style["left"] = "15px"
        self.fm = gui.CheckBoxLabel("Final Mix", checked=False, width="90px", height="auto")
        self.fm.style["position"] = "absolute"
        self.fm.style["top"] = "20px"
        self.fm.style["left"] = "100px"
        self.encoding = gui.DropDown(width="100px", margin="5px")
        self.encoding.append(gui.DropDownItem("International"), 1)
        self.encoding.append(gui.DropDownItem("Japanese"), 0)
        encoding = gui.HBox([gui.Label("Encoding/Region:"), self.encoding], width="200px")
        encoding.style["position"] = "absolute"
        encoding.style["top"] = "20px"
        encoding.style["left"] = "200px"
        openb = gui.Button("Open", width="70px", height="30px")
        openb.onclick.do(self.open)
        openb.style["position"] = "absolute"
        openb.style["top"] = "20px"
        openb.style["left"] = "430px"
        saveb = gui.Button("Save", width="70px", height="30px")
        saveb.onclick.do(self.save)
        saveb.style["position"] = "absolute"
        saveb.style["top"] = "20px"
        saveb.style["left"] = "530px"
        self.tabroot = gui.Container()
        self.tabroot.style["position"] = "relative"
        self.tabroot.style["top"] = "80px"
        self.root.append([slot, self.fm, encoding, openb, saveb, self.tabroot])

    def main(self):
        gui.DropDown.set_key = gui.DropDown.select_by_key # macro
        self.root = gui.Container(width="1000px", height="500px")
        self.root.style["position"] = "relative"
        self.menu_setup()
        return self.root

In [ ]:
server = remi.Server(KH1SaveEditor, title="KH1 Save Editor", address="0.0.0.0", port=8081, start_browser=False, start=False)
server.start()

### Old GUI

In [ ]:
class oKH1SaveEditor:
    def __init__(self):
        self.kh1 = KH1()

    def open(self, b=None):
        self.kh1 = KH1(self.slot.value, self.fm.value)
        self.parse()

    def save(self, b=None):
        if self.kh1 is not None:
            self.__save_playtime()
            self.__save_raft()
            self.kh1.difficulty.value = self.difficulty.value
            self.kh1.save()
        else:
            with self.output:
                self.output.clear_output()
                display("No loaded file to save!")

    def __save_playtime(self):
        if self.kh1.sysdata is not None:
            self.kh1.playtime.value = (self.hours.value * 3600 + self.minutes.value * 60 + self.seconds.value) * 60 + self.fraction.value
            print(self.kh1.playtime)

    def raft_validator(self, b=None):
        if self.encoding.value == 0:
            return self.raftjp_validator()
        if len(self.raft.value) > 9 or len(self.raft.value) == 0:
            self.raft.value = self.kh1.raft.decode("kh1us")
            self.valid_raft.value = "Invalid length, name restored from file!"
            return False
        else:
            self.valid_raft.value = "Valid!"
            self.raft.value = self.raft.value.encode("kh1us").decode("kh1us")
            return True

    # There are 2 byte characters that's why it's so different.
    def raftjp_validator(self):
        if len(bytearray(self.raftjp.value, "kh1jp")) > 10 or len(self.raftjp.value) == 0:
            self.raft.value = self.kh1.raft.decode("kh1jp")
            self.valid_raft.value = "Invalid length, name restored from file!"
            return False
        else:
            self.valid_raft.value = "Valid!"
            self.raft.value = self.raft.value.encode("kh1jp").decode("kh1jp")
            return True

    def __save_raft(self):
        if self.raft_validator():
            if self.encoding.value == 0:
                self.kh1.raft = bytearray(self.raft.value, "kh1jp") + bytearray(self.kh1.raft[len(bytearray(self.raft.value, "kh1jp")):])
            else:
                self.kh1.raft = bytearray(self.raft.value, "kh1us") + bytearray(self.kh1.raft[len(self.raft.value)+1:])
            return True
        else:
            return False

    def __parse_char(self, c: KH1Character):
        exp = widgets.BoundedIntText(value=c.exp.value,
                                     min=0,
                                     max=999999,
                                     step=1,
                                     description="EXP:",
                                     style={"description_width": "initial"},
                                     layout=widgets.Layout(width="240px"),
                                     disabled=False)
        level = widgets.BoundedIntText(value=c.level.value,
                                       min=1,
                                       max=100,
                                       step=1,
                                       description="Level:",
                                       style={"description_width": "initial"},
                                       layout=widgets.Layout(width="240px"),
                                       disabled=False)
        hp = widgets.BoundedIntText(value=c.hp.value,
                                    min=0,
                                    max=255,
                                    step=1,
                                    description="HP:",
                                    style={"description_width": "initial"},
                                    layout=widgets.Layout(width="240px"),
                                    disabled=False)
        maxhp = widgets.BoundedIntText(value=c.maxhp.value,
                                       min=0,
                                       max=255,
                                       step=1,
                                       description="Max HP:",
                                       style={"description_width": "initial"},
                                       layout=widgets.Layout(width="240px"),
                                       disabled=False)
        mp = widgets.BoundedIntText(value=c.mp.value,
                                    min=0,
                                    max=255,
                                    step=1,
                                    description="MP:",
                                    style={"description_width": "initial"},
                                    layout=widgets.Layout(width="240px"),
                                    disabled=False)
        maxmp = widgets.BoundedIntText(value=c.maxmp.value,
                                       min=0,
                                       max=255,
                                       step=1,
                                       description="Max MP:",
                                       style={"description_width": "initial"},
                                       layout=widgets.Layout(width="240px"),
                                       disabled=False)
        submp = widgets.HBox([widgets.BoundedIntText(value=c.submp.value // 30,
                                                     min=0,
                                                     max=mp.value,
                                                     step=1,
                                                     description="Sub MP:",
                                                     style={"description_width": "initial"},
                                                     disabled=False),
                              widgets.BoundedIntText(value=c.submp.value % 30,
                                                     min=0,
                                                     max=29,
                                                     step=1,
                                                     description=":",
                                                     style={"description_width": "initial"},
                                                     disabled=False)],
                              layout=widgets.Layout(width="244px"))
        maxap = widgets.BoundedIntText(value=c.maxap.value,
                                       min=0,
                                       max=255,
                                       step=1,
                                       description="Max AP:",
                                       style={"description_width": "initial"},
                                       layout=widgets.Layout(width="240px"),
                                       disabled=False)
        strength = widgets.BoundedIntText(value=c.strength.value,
                                          min=0,
                                          max=255,
                                          step=1,
                                          description="Strength:",
                                          style={"description_width": "initial"},
                                          layout=widgets.Layout(width="240px"),
                                          disabled=False)
        defense = widgets.BoundedIntText(value=c.defense.value,
                                         min=0,
                                         max=255,
                                         step=1,
                                         description="Defense:",
                                         style={"description_width": "initial"},
                                         layout=widgets.Layout(width="240px"),
                                         disabled=False)
        weapon = widgets.Dropdown(options=self.kh1.weapon_dict,
                                  value=c.weapon.value,
                                  description="Weapon:",
                                  style={"description_width": "initial"},
                                  layout=widgets.Layout(width="240px"),
                                  disabled=False)
        ability = widgets.VBox([widgets.HBox([widgets.Checkbox(value=c.abilities[i] & (1 << 7) == 0,
                                                               description="",
                                                               indent=False,
                                                               layout=widgets.Layout(width="30px"),
                                                               disabled=False),
                                              widgets.Dropdown(options=self.kh1.ability_dict,
                                                               value=c.abilities[i] & ~(1 << 7),
                                                               indent=False,
                                                               disabled=False)]) for i in range(48)])
        magic = widgets.VBox([widgets.Checkbox(value=bool(c.magic.value & (1 << i)),
                                               description=self.kh1.magicnames[i],
                                               indent=False,
                                               style={"description_width": "initial"},
                                               layout=widgets.Layout(width="90px"),
                                               disabled=False) for i in range(7)])
        return widgets.HBox([widgets.VBox([weapon, exp, level, hp, maxhp, mp, maxmp, submp, maxap, strength, defense, magic]),
                             self.empty, self.empty, ability])

    def parse_playtime(self, rawtime):
        time = widgets.HBox([])
        playtime = rawtime.value // 60
        hours = playtime // 3600
        minutes = (playtime % 3600) // 60
        seconds = (playtime % 3600) % 60
        fraction = rawtime.value % 60
        hours = widgets.BoundedIntText(value=hours,
                                       min=0,
                                       max=99,
                                       step=1,
                                       description="Playtime:",
                                       style={"description_width": "initial"},
                                       layout=widgets.Layout(width="120px"),
                                       disabled=False)
        minutes = widgets.BoundedIntText(value=minutes,
                                         min=0,
                                         max=59,
                                         step=1,
                                         description=":",
                                         style={"description_width": "initial"},
                                         layout=widgets.Layout(width="50px"),
                                         disabled=False)
        seconds = widgets.BoundedIntText(value=seconds,
                                         min=0,
                                         max=59,
                                         step=1,
                                         description=":",
                                         style={"description_width": "initial"},
                                         layout=widgets.Layout(width="50px"),
                                         disabled=False)
        fraction = widgets.BoundedIntText(value=fraction,
                                          min=0,
                                          max=59,
                                          step=1,
                                          description="::",
                                          style={"description_width": "initial"},
                                          layout=widgets.Layout(width="55px"),
                                          disabled=False)
        time = widgets.HBox([hours, minutes, seconds, fraction]), hours, minutes, seconds, fraction
        return time

    def __parse_general(self):
        self.curve = widgets.Dropdown(options=[("Dawn", 0), ("Midday", 1), ("Dusk", 2)],
                                      value=self.kh1.curve.value,
                                      description="Leveling Curve:",
                                      style={"description_width": "initial"},
                                      disabled=False)
        self.path = widgets.Dropdown(options=[("Warrior", 0), ("Guardian", 1), ("Mystic", 2)],
                                            value=self.kh1.path.value,
                                            description="Path:",
                                            style={"description_width": "initial"},
                                            disabled=False)
        world_dict = [(v, k) for k, v in self.kh1.world_dict.items()]
        self.world = widgets.Dropdown(options=world_dict,
                                      value=self.kh1.world.value,
                                      description="World:",
                                      style={"description_width": "initial"},
                                      disabled=False)
        self.room = widgets.BoundedIntText(value=self.kh1.room.value,
                                           min=0,
                                           max=255,
                                           step=1,
                                           description="Room:",
                                           style={"description_width": "initial"},
                                           disabled=False)
        self.flag = widgets.BoundedIntText(value=self.kh1.flag.value,
                                           min=0,
                                           max=255,
                                           step=1,
                                           description="Flag:",
                                           style={"description_width": "initial"},
                                           disabled=False)
        self.party = widgets.HBox([widgets.Dropdown(options=self.kh1.character_dict,
                                           value=self.kh1.party[0],
                                           description="Party:",
                                           style={"description_width": "initial"},
                                           disabled=True),
                                   widgets.Dropdown(options=self.kh1.character_dict,
                                           value=self.kh1.party[1],
                                           description="Friend 1:",
                                           style={"description_width": "initial"},
                                           disabled=False),
                                   widgets.Dropdown(options=self.kh1.character_dict,
                                           value=self.kh1.party[2],
                                           description="Friend 2:",
                                           style={"description_width": "initial"},
                                           disabled=False),
                                   widgets.Dropdown(options=self.kh1.character_dict,
                                           value=self.kh1.party[3],
                                           description="Friend 3:",
                                           style={"description_width": "initial"},
                                           disabled=False)])
        self.playtime = widgets.HBox([])
        if self.kh1.sysdata is not None:
            self.playtime, self.hours, self.minutes, self.seconds, self.fraction = self.parse_playtime(self.kh1.playtime)
        self.general = widgets.VBox([self.playtime,
                                     widgets.HBox([self.curve, self.empty, self.path]),
                                     widgets.HBox([self.world, self.room, self.flag]),
                                     self.party])

    def __parse_characters(self, characters):
        # I leave out Pooh because of his invalid abilities for now.
        characters.children = [self.__parse_char(c) for c in self.kh1.characters if c.name != "Winnie the Pooh"]
        characters.titles = [c.name for c in self.kh1.characters if c.name != "Winnie the Pooh"]
        characters.layout = widgets.Layout(width="100%")

    def __parse_journal(self):
        self.journal_unlocked = widgets.Checkbox(value=bool(self.kh1.journal_unlock.value & (1 << 3)),
                                                 description="Journal unlocked",
                                                 style={"description_width": "initial"},
                                                 indent=False,
                                                 disabled=False)
        self.journal_tabs = widgets.Tab()
        heartless = list(self.kh1.heartless)
        heartlessnames = self.kh1.heartlessnames
        if self.kh1.fm:
            heartless += list(self.kh1.fmheartless)
            heartlessnames = self.kh1.fmheartlessnames
        self.journal_heartless = widgets.VBox([widgets.BoundedIntText(value=heartless[i],
                                                                      min=0,
                                                                      max=0xFFFF,
                                                                      description=heartlessnames[i] + ":",
                                                                      style={"description_width": "initial"},
                                                                      layout=widgets.Layout(width="200px"),
                                                                      disabled=False) for i in range(len(heartless)) if "Unused" not in heartlessnames[i]])
        self.journal_tabs.children = [self.journal_heartless]
        self.journal_tabs.titles = ["The Heartless"]
        self.journal = widgets.VBox([self.journal_unlocked, self.journal_tabs])

    def __parse_inventory(self):
        self.inventory = widgets.HBox([widgets.VBox([widgets.BoundedIntText(value=self.kh1.inventory[i],
                                                                            min=0,
                                                                            max=255,
                                                                            description=list(self.kh1.item_dict.keys())[i] + ":",
                                                                            style={"description_width": "initial"},
                                                                            layout=widgets.Layout(width="170px"),
                                                                            disabled=False) for i in range(1, 0x48) if "Unused" not in list(self.kh1.item_dict.keys())[i]]),
                                       #widgets.VBox([widgets.BoundedIntText(value=self.kh1.inventory[i],
                                       #                                     min=0,
                                       #                                     max=255,
                                       #                                     description=list(self.kh1.item_dict.keys())[i] + ":",
                                       #                                     style={"description_width": "initial"},
                                       #                                     layout=widgets.Layout(width="170px"),
                                       #                                     disabled=False) for i in range(0x48, 0x51)]),
                                       widgets.VBox([widgets.BoundedIntText(value=self.kh1.inventory[i],
                                                                            min=0,
                                                                            max=255,
                                                                            description=list(self.kh1.item_dict.keys())[i] + ":",
                                                                            style={"description_width": "initial"},
                                                                            layout=widgets.Layout(width="170px"),
                                                                            disabled=False) for i in range(0x51, 0x8E) if "Unused" not in list(self.kh1.item_dict.keys())[i]]),
                                       widgets.VBox([widgets.BoundedIntText(value=self.kh1.inventory[i],
                                                                            min=0,
                                                                            max=255,
                                                                            description=list(self.kh1.item_dict.keys())[i] + ":",
                                                                            style={"description_width": "initial"},
                                                                            layout=widgets.Layout(width="170px"),
                                                                            disabled=False) for i in range(0x8E, 0xC8) if "Unused" not in list(self.kh1.item_dict.keys())[i]]),
                                       widgets.VBox([widgets.BoundedIntText(value=self.kh1.inventory[i],
                                                                            min=0,
                                                                            max=255,
                                                                            description=list(self.kh1.item_dict.keys())[i] + ":",
                                                                            style={"description_width": "initial"},
                                                                            layout=widgets.Layout(width="170px"),
                                                                            disabled=False) for i in range(0xC8, 256) if "Unused" not in list(self.kh1.item_dict.keys())[i]])])

    def __parse_config(self):
        self.autolock = widgets.Dropdown(options=[("On", 0), ("Off", 1)],
                                         value=self.kh1.autolock.value,
                                         description="Auto Lock:",
                                         style={"description_width": "initial"},
                                         disabled=False)
        self.autolock.observe(lambda change: setattr(self.kh1.autolock, "value", change["new"]), names="value")
        self.targetlock = widgets.Dropdown(options=[("Auto", 0), ("Manual", 1)],
                                         value=self.kh1.targetlock.value,
                                         description="Target Lock:",
                                         style={"description_width": "initial"},
                                         disabled=False)
        self.camera = widgets.Dropdown(options=[("Auto", 0), ("Manual", 1)],
                                         value=self.kh1.camera.value,
                                         description="Camera:",
                                         style={"description_width": "initial"},
                                         disabled=False)
        self.vibration = widgets.Dropdown(options=[("On", 0), ("Off", 1)],
                                          value=self.kh1.vibration.value,
                                          description="Vibration:",
                                          style={"description_width": "initial"},
                                          disabled=False)
        self.sound = widgets.Dropdown(options=[("Stereo", 0), ("Mono", 1)],
                                      value=self.kh1.sound.value,
                                      description="Sound:",
                                      style={"description_width": "initial"},
                                      disabled=False)
        self.datainstall = widgets.Dropdown(options=[("DVD-ROM", 0), ("Hard Drive", 1)],
                                            value=self.kh1.datainstall.value,
                                            description="Data Install:",
                                            style={"description_width": "initial"},
                                            disabled=not self.kh1.fm and bool(self.encoding.value))
        if self.kh1.fm:
            self.difficulty = widgets.Dropdown(options=[("Beginner", 0), ("Standard", 1), ("Proud", 2)],
                                               value=self.kh1.difficulty.value,
                                               description="Difficulty:",
                                               style={"description_width": "initial"},
                                               disabled=False)
            if self.__f1 in self.encoding._trait_notifiers["value"]["change"]:
                self.encoding.unobserve(self.__f1, names="value")
                self.encoding.unobserve(self.__f2, names="value")
                self.difficulty.unobserve(self.__f3, names="value")
                self.datainstall.unobserve(self.__f4, names="value")
        else:
            self.difficulty = widgets.Dropdown(options=[("Normal", 0), ("Expert", 1)],
                                               value=self.kh1.difficulty.value,
                                               description="Difficulty:",
                                               style={"description_width": "initial"},
                                               ensure_option=False,
                                               disabled=not bool(self.encoding.value))
            self.encoding.observe(self.__f1, names="value")
            self.encoding.observe(self.__f2, names="value")
            self.difficulty.observe(self.__f3, names="value")
            self.datainstall.observe(self.__f4, names="value")
        self.config = widgets.VBox([self.autolock, self.targetlock, self.camera,
                                    self.vibration, self.sound,
                                    self.datainstall, self.difficulty])
        

    def __parse_misc(self):
        self.magiclevels = widgets.VBox([widgets.Dropdown(options=[(self.kh1.magicnames[i], 1),
                                                                   (self.kh1.magicnames2[i], 2),
                                                                   (self.kh1.magicnames3[i], 3)],
                                                          value=self.kh1.magiclevels[i],
                                                          description=f"{self.kh1.magicnames[i]} level:",
                                                          style={"description_width": "initial"},
                                                          disabled=False) for i in range(7)])
        self.raft = widgets.Text(description="Raft name:",
                                 value=self.kh1.raft.decode(["kh1jp", "kh1us"][self.encoding.value]),
                                 style={"description_width": "initial"},
                                 layout=widgets.Layout(width="200px"))
        self.raftjp = widgets.Text(description="JP Raft name:",
                                   value=self.kh1.raft.decode("kh1jp"),
                                   style={"description_width": "initial"},
                                   layout=widgets.Layout(width="200px"))
        self.valid_raft = widgets.Label("")
        raft_validator = widgets.Button(description="Validate")
        raft_validator.on_click(self.raft_validator)
        self.misc = widgets.VBox([widgets.HBox([self.magiclevels]),
                                  widgets.HBox([self.raft, raft_validator, self.valid_raft])])

    def parse(self):
        self.tabs = widgets.Tab()
        self.characters = widgets.Tab()
        self.__parse_general()
        self.__parse_characters(self.characters)
        self.__parse_inventory()
        self.__parse_journal()
        self.__parse_config()
        self.__parse_misc()
        self.tabs.children = [self.general, self.characters, self.inventory, self.journal, self.config, self.misc]
        self.tabs.titles = ["General", "Characters", "Inventory", "Journal", "Config", "Misc"]
        with self.output:
            self.output.clear_output()
            display(self.tabs)

    def menu_setup(self):
        self.slot = widgets.BoundedIntText(value=1,
                                           min=1,
                                           max=99,
                                           step=1,
                                           description="Slot:",
                                           style={"description_width": "initial"},
                                           indent=False,
                                           layout=widgets.Layout(width="75px"),
                                           disabled=False)
        self.fm = widgets.Checkbox(value=False,
                                   description="Final Mix",
                                   style={"description_width": "initial"},
                                   indent=False,
                                   layout=widgets.Layout(width="80px"),
                                   disabled=False)
        self.encoding = widgets.Dropdown(options=[("International", 1), ("Japanese", 0)],
                                         value=1,
                                         description="Encoding/Region:",
                                         style={"description_width": "initial"},
                                         layout=widgets.Layout(width="220px"),
                                         disabled=False)
        self.encoding.observe(lambda change: setattr(self.raft, "value", self.kh1.raft.decode(["kh1jp", "kh1us"][change["new"]])),
                              names="value")
        self.__f1 = lambda change: setattr(self.difficulty, "disabled", [True, False][change["new"]])
        self.__f2 = lambda change: setattr(self.datainstall, "disabled", [False, True][change["new"]])
        self.__f3 = lambda change: setattr(self.datainstall, "value", change["new"])
        self.__f4 = lambda change: setattr(self.difficulty, "value", change["new"])
        openb = widgets.Button(description="Open", style={"description_width": "initial"})
        openb.on_click(self.open)
        saveb = widgets.Button(description="Save", style={"description_width": "initial"})
        saveb.on_click(self.save)
        self.empty = widgets.Label("", layout=widgets.Layout(width="10px"))
        row1 = widgets.HBox([self.slot, self.empty, self.fm, self.empty, self.encoding])
        row2 = widgets.HBox([openb, self.empty, saveb])
        self.menu = widgets.VBox([row1, row2])

    def initialize(self):
        self.menu_setup()
        self.output = widgets.Output()
        warning = widgets.HTML("<p>Vanilla save files' names must be in USA format:<br><b>BASLUS-20370-01</b><br>Final Mix format:<br><b>BISLPS-25198-01</b><br>The last 2 digits are the slot number.</p>")
        self.editor = widgets.VBox([warning, self.menu, self.output])
        return self.editor


In [ ]:
kh1 = oKH1SaveEditor()
kh1.initialize()

In [ ]:
#kh1.open()
#kh1.save()

In [ ]:
import json, re

class HexIntEncoder(json.JSONEncoder):
    def __init__(self, *args, **kwargs):
        # allow non-ASCII characters (preserves apostrophes)
        kwargs['ensure_ascii'] = False
        super().__init__(*args, **kwargs)

    def encode(self, o):
        # get the normal JSON string
        s = super().encode(o)
        # regex-replace every standalone decimal number with its hex repr
        return re.sub(r'\b(\d+)\b', lambda m: f"0x{int(m.group(1)):02X}", s)

inv = {v: k for k, v in kh1.kh1.ability_dict.items()}

print(json.dumps(inv, indent=4, cls=HexIntEncoder))

## KH2FM

In [ ]:
%%writefile kh2fm_checksum.cpp
#include <iostream>
#include <fstream>
using namespace std;

#define uint unsigned int
#define byte unsigned char

int CrcPolynomial = 0x04c11db7;
uint CalculateChecksum(byte data[], int offset, int length, unsigned int checksum);
uint crc_table[0x100] = {0};
uint MaxValue = 0xFFFFFFFF;
byte header[8] = {'K', 'H', '2', 'J', 0x3A, 0x00, 0x00, 0x00};

void GetCrcTable(int polynomial)
{
    for (int x = 0; x < 0x100; x++)
    {
        int r = x << 24;
        for (int j = 0; j < 0xff; j++)
        r = r << 1 ^ (r < 0 ? polynomial : 0);
        crc_table[x] = (uint)r;
        //printf("%08X\n", crc_table[x]);
    }
}

uint CalculateChecksum(byte data[], int offset, int length, uint checksum)
{
    for (int i = offset; i < offset + length; i++)
        checksum = crc_table[(checksum >> 24) ^ data[i]] ^ (checksum << 8);

    return checksum ^ MaxValue;
}

int main(int argc, char* argv[])
{
    if (*argv[1] <= '0' || *argv[1] > '9')
    {
        printf("Give me a valid slot!");
        return 1;
    }

    byte data[0x10FC0] = {0};
    GetCrcTable(CrcPolynomial);

    int slot = atoi(argv[1]) - 1;
    char fajlnev[20];
    sprintf(fajlnev, "BISLPM-66675FM-%02d", slot);
    printf("%s\n", fajlnev);
    fstream fajl(fajlnev);
    if (fajl.fail())
    {
        printf("No file or something's went wrong.");
        return 1;
    }

    for (int i = 0; i < 0x10FC0; i++)
    {
        data[i] = fajl.get();
    }

    uint checksum = CalculateChecksum(data, 0, 8, MaxValue);
    printf("%08X\n", checksum);

    checksum = CalculateChecksum(data, 0xC, 0x10FB4, checksum ^ MaxValue);
    printf("%04X\n", checksum);
    printf("\n");

    fajl.seekp(0x8);
    fajl.write((char*)&checksum,  sizeof(checksum));
    fajl.close();

//    system("PAUSE");
    return 0;
}

In [ ]:
! g++ kh2fm_checksum.cpp -o kh2fm_checksum
! ./kh2fm_checksum 1

## KHBBSFM

In [ ]:
%%writefile khbbsfm_checksum.cpp
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <iostream>
#include <fstream>
#include <iomanip>
using namespace std;

typedef unsigned int uint;
typedef unsigned char byte;


uint CalculateChecksum(byte data[], int offset, int length);

uint CalculateChecksum(byte data[], int offset, int length)
{
    uint checksum = 0;
    for (int i = offset; i < (offset + length); i+=4)
        checksum += *(uint*)&data[i];

    return checksum;
}

int main()
{
    byte data[0x11E50] = {0};

    fstream fajl("SAVEDATA.DAT");
    if (fajl.fail())
    {
        printf("No file or something's went wrong.");
        return 1;
    }

    for (int i = 0; i < 0x11E50; i++)
    {
        data[i] = fajl.get();
        //printf("%02X: %02X\n", i, data[i]);
    }

//    uint checksum = *(uint*)(data+0xC);
//    printf("%08X\n", checksum);

    printf("SAVEDATA.DAT\n");
    uint checksum = CalculateChecksum(data, 0x10, 0x11E40);
    printf("%08X\n", checksum);
    printf("\n");

    fajl.seekp(0xC);
    fajl.write((char*)&checksum,  sizeof(checksum));
    fajl.close();

//    system("PAUSE");
    return 0;
}

In [ ]:
! g++ khbbsfm_checksum.cpp -o khbbsfm_checksum
! ./khbbsfm_checksum

In [ ]:
class KHBBSFM:
    def __init__(self):
        self.readfile("SAVEDATA.DAT")

    def readfile(self, filename):
        with open(filename, "rb") as file:
            self.data = (c_ubyte*0x16CC0)(*file.read())


## KHReCoM

In [ ]:
%%writefile khrecom_checksum.cpp
#include <iostream>
#include <fstream>
using namespace std;

#define uint unsigned int
#define byte unsigned char

int CrcPolynomial = 0x04c11db7;
uint CalculateChecksum(byte data[], int offset, int length);

uint CalculateChecksum(byte data[], int offset, int length)
{
    int checksum = 0xFFFFFFFF;

    for (int i = offset; i < offset+length; i++)
    {
        checksum ^= data[i] << 31;
        checksum = checksum << 1 ^ (checksum < 0 ? 0x4c11db7 : 0);
    }

    return (uint)~checksum;
}

int main(int argc, char* argv[])
{
    if (*argv[1] <= '0' || *argv[1] > '9')
    {
        printf("Give me a valid slot!");
        return 1;
    }

    byte data[0x3630] = {0};

    int slot = atoi(argv[1]);
    if (slot > 99)
    {
        printf("Slot must be between 1 and 99!");
        return 1;
    }
    char fajlnev[20];
    sprintf(fajlnev, "BASLUS-21799COM-%02d", slot);
    printf("%s\n", fajlnev);
    fstream fajl(fajlnev);
    if (fajl.fail())
    {
        printf("No file or something's went wrong.");
        return 1;
    }

    for (int i = 0; i < 0x3630; i++)
    {
        data[i] = fajl.get();
        //printf("%02X: %02X\n", i, data[i]);
    }

    uint checksum = *(uint*)(data+0x4);
    printf("%08X\n", checksum);

    checksum = CalculateChecksum(data, 0x10, 0x3620);
    printf("%08X\n", checksum);
    printf("\n");

    fajl.seekp(0x4);
    fajl.write((char*)&checksum,  sizeof(checksum));
    fajl.close();

    //system("PAUSE");
    return 0;
}

In [ ]:
! g++ khrecom_checksum.cpp -o khrecom_checksum
! ./khrecom_checksum 1